# Canal axi

In [ ]:
from trustutils.jupyter import run

run.introduction("Yannick Gorsse")
run.description("Driven cavity with cosine thermal power and thermal exchange with uniform external temperature")
run.TRUST_parameters()

In [ ]:
from trustutils.jupyter import run
import os

run.reset()
problems = [("ns", "-+"), ("pbmulti", "-x")]
flux = 1e5
r = 0.1
h = 1.0
for pb, _ in problems:
    for axi, nom_axi in [("", "2D"), ("bidim_axi", "2D_axi")]:
        run.addCaseFromTemplate(f"jdd_{pb}.data", f"{pb}/{nom_axi}", {"flux" : flux, "r" : r, "h" : h, "h_2" : h / 2.0, "axi" : axi})

run.printCases()
run.runCases()
run.tablePerf()

## Pas de temps

In [ ]:
from trustutils import plot
import numpy as np

a = plot.Graph("Time steps")

for pb, marker in problems:
    for nom_axi in ["2D", "2D_axi"]:
        dt_ev = np.loadtxt(f"{run.BUILD_DIRECTORY}/{pb}/{nom_axi}/jdd_{pb}.dt_ev")
        a.add(dt_ev[:, 0], dt_ev[:, 1], label=f"{pb} {nom_axi}", marker=marker)

## Profils

In [ ]:
vars = ("V", "T")
a = plot.Graph("", nY=len(vars))
for j, var in enumerate(vars):
    a.addPlot(j, f"Variable {var}")
    for pb, marker in problems:
        for nom_axi, color in [("2D", "red"), ("2D_axi", "black")]:
            a.addSegment(f"{run.BUILD_DIRECTORY}/{pb}/{nom_axi}/jdd_{pb}_{var}.son", label=f"{pb} {nom_axi}", compo=(1 if var == "V" else 0), marker=marker, color=color, markersize=15)


## Puissance

In [ ]:
import pandas as pd
from math import pi

for nom_axi, color in [("2D", "red"), ("2D_axi", "black")]:
    for pb, marker in problems:
        df = pd.read_csv(f"{run.BUILD_DIRECTORY}/{pb}/{nom_axi}/jdd_{pb}_pbt_Diffusion_chaleur.out", comment="#", delim_whitespace=True)
        print(f"{pb:8s} {nom_axi:6s} {df['wall'].to_list()[-1]:.1f}")
print(f"Analytique      {flux * h * 2.0 * pi * r:.1f}")